In [16]:
from datasets import load_dataset
data = load_dataset("Sing0402/fever_200")["train"]

In [20]:
for idx, instance in enumerate(data):


    question = instance["question"]
    answer = instance["answer"]

In [21]:
question

'What part of an ar 15 has the serial number?'